In [ ]:
# @title Install yt-dlp (preferred tool) and update ffmpeg
!pip install -U yt-dlp
!apt-get update && apt-get install -y ffmpeg

# Verify installation
import yt_dlp
import subprocess
print(f"yt-dlp version: {yt_dlp.version.__version__}")
print(f"ffmpeg version: {subprocess.check_output(['ffmpeg', '-version']).decode().split()[2]}")

In [ ]:
# @title Step 1: Input and List formats

from IPython.display import clear_output

# 1. Input the URL
video_url = "https://youtu.be/oyZCJaUBb1w" # @param {type:"string"}

# 2. Configure yt-dlp to list formats only
ydl_opts = {
    'quiet': False,
    'no_warnings': False,
    'listformats': True, # This triggers the format list output
}

print("\n--- Fetching Format List ---\n")
try:
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.extract_info(video_url, download=False)
except Exception as e:
    print(f"Error fetching info: {e}")

print("\n--- Instructions ---")
print("Look at the table above.")
print("Find the CODE for the video resolution you want (e.g., 135, 136, 137).")
print("Find the CODE for the audio quality you want (usually 140 for m4a).")
print("You will enter them in the next cell like: 135+140")

In [ ]:
# @title Step 2: Select Format IDs
# @markdown Enter the Video ID and Audio ID separated by a plus sign (e.g., `135+140`).
# @markdown **Important:** Ensure the type is set to string so math isn't performed.
format_selection = "135+140" # @param {type:"string"}

# Force conversion to string just in case Colab still tries to be smart
selected_format = str(format_selection)

if '+' not in selected_format:
    print(f"⚠️ Warning: You entered '{selected_format}'. It doesn't look like 'VideoID+AudioID'.")
    print("yt-dlp might pick default formats automatically.")
else:
    print(f"✅ Selected format combination: {selected_format}")

In [ ]:
# @title Step 3: Optional Time Range (Trimming)
# @markdown Leave both fields empty to download the **full video**.
# @markdown Format: `HH:MM:SS` (e.g., `00:05:00` for 5 minutes in).
start_time = "" # @param {type:"string"}
end_time = "" # @param {type:"string"}

# Logic to prepare the argument for yt-dlp --download-sections
# Syntax required by yt-dlp: *START-END (e.g., *00:01:00-00:02:00)
section_arg = None

if start_time or end_time:
    # If start is empty, default to 0. If end is empty, default to 'inf' (infinity)
    safe_start = start_time if start_time else "0"
    safe_end = end_time if end_time else "inf"

    section_arg = f"*{safe_start}-{safe_end}"
    print(f"✅ Trimming configured:")
    print(f"   Start: {safe_start}")
    print(f"   End:   {safe_end}")
    print(f"   Internal Arg: {section_arg}")
else:
    print("✅ No time range selected. Downloading full video.")

# This variable will be used in Cell 5
download_section_flag = section_arg

In [ ]:
# @title Step 4: Execute Download (Dynamic Filename)
# @markdown This cell uses the variables defined in Steps 1, 2, and 3.
# @markdown The output file will be named automatically based on the Video Title.

import os
import subprocess
import glob

# --- Configuration from Previous Cells ---
# video_url (from Cell 2)
# selected_format (from Cell 3)
# download_section_flag (from Cell 4)

# Define Output Template
# %(title)s : Uses the video title
# %(ext)s   : Uses the correct extension after merging (mp4)
# We sanitize the title automatically via yt-dlp to avoid invalid filename characters
output_template = "%(title)s.%(ext)s"

print(f"🚀 Starting Download...")
print(f"   URL: {video_url}")
print(f"   Format: {selected_format}")
if download_section_flag:
    print(f"   Section: {download_section_flag}")
else:
    print(f"   Section: Full Video")
print(f"   Output Name Pattern: {output_template}")

# --- Construct the yt-dlp Command ---
cmd = [
    "yt-dlp",
    "-f", selected_format,
    "--merge-output-format", "mp4",
    "-o", output_template,  # Dynamic filename here
]

# Add the modern --download-sections flag if a time range was specified
if download_section_flag:
    cmd.extend(["--download-sections", download_section_flag])

# Add the URL last
cmd.append(video_url)

# --- Execute Command ---
try:
    # Run the command
    result = subprocess.run(cmd, check=True, text=True, capture_output=False)

    # Find the generated file
    # Since we don't know the exact title beforehand, we look for the most recent .mp4 file
    # that matches the pattern created in this session.
    list_of_files = glob.glob("*.mp4")

    if list_of_files:
        # Get the most recently modified file
        latest_file = max(list_of_files, key=os.path.getctime)
        file_size_mb = os.path.getsize(latest_file) / (1024 * 1024)

        print("\n" + "="*30)
        print(f"✅ SUCCESS!")
        print(f"   Saved as: {latest_file}")
        print(f"   Size: {file_size_mb:.2f} MB")
        print("="*30)
        print("💡 You can find the file in the folder icon (📁) on the left sidebar.")
    else:
        print("\n❌ Error: Download finished but no .mp4 file was found in the directory.")

except subprocess.CalledProcessError as e:
    print(f"\n❌ Download failed with error code: {e.returncode}")
    print("Check the logs above for specific yt-dlp errors.")
except Exception as e:
    print(f"\n❌ An unexpected error occurred: {str(e)}")